In [9]:
"""
    Instruções:
        1- Download dos dados municipais do estado de Minas Gerais;
        2- Transformação dos dados de desmatamento;
        3- Processamento dos dados:
            -reprojeção para EPSG:31983;
            -cálculo de área;
        4- Geração dos arquivos de saída no formato GeoJSON;
"""

'\n    Instruções:\n        1- Download dos dados municipais do estado de Minas Gerais;\n        2- Transformação dos dados de desmatamento;\n        3- Processamento dos dados:\n            -reprojeção para EPSG:31983;\n            -cálculo de área;\n        4- Geração dos arquivos de saída no formato GeoJSON;\n'

In [13]:
import geopandas as gpd 
import pandas as pd
import requests
pd.set_option('display.max_columns', None)

In [3]:
#Fazendo o get do JSON dos munincipios de MG, e transformando ele em geojson
url = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json'
response = requests.get(url)
data = response.json()
gdf = gpd.GeoDataFrame.from_features(data["features"])
#Dentro do arquivo não há evidencias do elipsoide de referencia, então vou considerar o padrão WGS84
gdf.crs = "EPSG:4326"
#Fazendo a conversão para EPSG==31983
gdf = gdf.to_crs(epsg=31983)

In [10]:
#Feature engineering: Criando uma novo coluna para a área total do município em km**2
gdf["area_km2"] = gdf.geometry.area / 1e6
#Salvando o arquivo com o nome especificado
output_file = "dados/municipios-mg.geojson"
gdf.to_file(output_file, driver="GeoJSON")

print(f"Arquivo salvo em {output_file}")

Arquivo salvo em dados/municipios-mg.geojson


In [26]:
#Agora vamos fazer a call dos arquivos de desmatamento e vamos uni-los
file_path_ago22 = r"dados/desmatamento_ago22.gpkg"
file_path_set22 = r"dados/desmatamento_set22.gpkg"
# Lê os arquivos GeoPackage
desmatamento_ago22 = gpd.read_file(file_path_ago22)
desmatamento_set22 = gpd.read_file(file_path_set22)
# Junta os GeoDataFrames
focos_desmatamento = gpd.GeoDataFrame(pd.concat([desmatamento_ago22, desmatamento_set22], ignore_index=True))
# Converte para a projeção EPSG:31983
focos_desmatamento = focos_desmatamento.to_crs(epsg=31983)
# Salva o GeoDataFrame no diretório 'dados' com o nome 'focos-desmatamento-mg.geojson'
output_file = "dados/focos-desmatamento-mg.geojson"
focos_desmatamento.to_file(output_file, driver="GeoJSON")

print(f"Arquivo salvo em {output_file}")

Arquivo salvo em dados/focos-desmatamento-mg.geojson


In [31]:
#Agora vamos fazer a call do XLSX do PIB dos municipios Brasileiros, vamos selecionar o ano que mais se aproxima de 2022 (2021 no caso), e selecionar somente MG como estado
#(fonte: https://www.ibge.gov.br/estatisticas/economicas/contas-nacionais/9088-produto-interno-bruto-dos-municipios.html?=&t=downloads)
pib_brasil = pd.read_excel(r"dados\PIB dos Municípios - base de dados 2010-2021.xlsx")
#Filtrando para 2021 e para o estado de MG somente
pib_mg_2021 = pib_brasil[(pib_brasil['Ano'] == 2021) & (pib_brasil['Nome da Unidade da Federação'] == 'Minas Gerais')]
#Salvando para CSV
pib_mg_2021.to_csv(r"dados/populacao-pib-municipios-mg.csv")


